## Analysis of scraped data form KV.ee

### Import of packages

In [1]:
import os # this is needed atleast for me because otherwise reprojection of coordinate systems does not work. 
os.environ['PROJ_DATA'] = r'C:\Users\Dan\micromamba\envs\geopython2025\Library\share\proj'
os.environ['PROJ_LIB'] = r'C:\Users\\Dan\micromamba\envs\geopython\Library\share\proj'

In [2]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt
from shapely.ops import nearest_points
from tqdm import tqdm
import numpy as np
import matplotlib.patches as mpatches

In [3]:
import mapclassify
import matplotlib.colors as mcolors
from mapclassify import NaturalBreaks
import contextily as ctx
from networkx import ego_graph
from shapely.geometry import box
from itertools import chain
import seaborn as sns
import matplotlib

In [4]:
# spatial libs
import libpysal
from libpysal.weights import KNN, DistanceBand
from esda.moran import Moran
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
from esda.moran import Moran_BV
from libpysal.weights import KNN
from spreg import ML_Lag, ML_Error

In [6]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt

# Data setup

In [ ]:
tqdm.pandas()

In [ ]:
# Tartu only for now
df = pd.read_csv("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_csvs/listings_cleaned_tartu.csv", sep=";") 
gdf_rent = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
gdf_rent = gdf_rent.to_crs(epsg=3301) # reprojecting the data into the Estonian national coordinate system. So from a geographic coordinate system to a projected coordinate system


In [ ]:
#gdf_rent

## OSM download

### OSM roads downlad

In [ ]:
# Defining the study area
place_name = "Tartu, Estonia"
gdf_place = ox.geocode_to_gdf(place_name) # queries OpenStreetMap service to find the geographical boundary polygon of Tartu city not including the rural areas such as former Tähtvere municipality
polygon = gdf_place.geometry.iloc[0] # extracts the polygon

In [ ]:
# Downloading the streetnetwork form OpenStreetMap
G = ox.graph_from_polygon(polygon, network_type='walk') # downloads the street network that is inside the polygon
G = ox.project_graph(G, to_crs='EPSG:3301')  # reprojecting the data into the Estonian national coordinate system 

#### OSM services download

In [ ]:
# Getting all the tags that are classified as amenities form OpenStreetmaps data that are inside the study area
tags_amenity = {"amenity": True}
amenity = ox.features_from_polygon(polygon, tags_amenity)
amenity = amenity.to_crs(gdf_rent.crs)

In [ ]:
# Since some of these POI geometries are points and some are polygons we will split them for cleaning
amenity_points = amenity[amenity.geometry.type == "Point"]
amenity_polygons = amenity[amenity.geometry.type.isin(["Polygon", "MultiPolygon"])]

In [ ]:
# Turning polygon geometries into centroids
amenity_polygons_centroids = amenity_polygons.copy()
amenity_polygons_centroids["geometry"] = amenity_polygons_centroids.geometry.centroid

In [ ]:
# Combining the data that were points and the data that were polygons but were turned into points
amenity_combined = pd.concat([amenity_points, amenity_polygons_centroids])
amenity_combined = amenity_combined.reset_index(drop=True)

In [ ]:
# Getting all the tags that are classified as shops and leisure form OpenStreetmaps data that are inside the study area
tags_shop = {"shop": True}
tags_leisure = {"leisure": True}
shops = ox.features_from_polygon(polygon, tags_shop)
leisure = ox.features_from_polygon(polygon, tags_leisure)
# Matching the CRS (coordinate referece system)
shops = shops.to_crs(gdf_rent.crs)
leisure = leisure.to_crs(gdf_rent.crs)

In [ ]:
# Shops
shops_points = shops[shops.geometry.type == "Point"]
shops_polygons = shops[shops.geometry.type.isin(["Polygon", "MultiPolygon"])]

# Leisure
leisure_points = leisure[leisure.geometry.type == "Point"]
leisure_polygons = leisure[leisure.geometry.type.isin(["Polygon", "MultiPolygon"])]

In [ ]:
# Shops
shops_polygons_centroids = shops_polygons.copy()
shops_polygons_centroids["geometry"] = shops_polygons_centroids.geometry.centroid

# Leisure
leisure_polygons_centroids = leisure_polygons.copy()
leisure_polygons_centroids["geometry"] = leisure_polygons_centroids.geometry.centroid

In [ ]:
# Shops
shops_combined = pd.concat([shops_points, shops_polygons_centroids]).reset_index(drop=True)

# Leisure
leisure_combined = pd.concat([leisure_points, leisure_polygons_centroids]).reset_index(drop=True)

#### Looking at the data

In [ ]:
# Get nodes and edges from the graph as GeoDataFrames
nodes, edges = ox.graph_to_gdfs(G)

# Create a base plot
fig, ax = plt.subplots(figsize=(10, 10))

# Optional: plot rental listings too
gdf_rent.plot(ax=ax, markersize=15, color='blue', alpha=0.6, label="Rent Listings")

# Plot the edges (streets)
edges.plot(ax=ax, linewidth=0.8, color='gray', label="Streets")

# Plot the nodes (intersections)
#nodes.plot(ax=ax, markersize=5, color='black', label="Nodes")

# Plot amenities (POIs)
#amenity_combined.plot(ax=ax, markersize=20, color='red', alpha=0.6, label="Amenities")



# Add legend and title
ax.set_title("Tartu: Street Network, Amenities, and Rental Listings", fontsize=14)
ax.legend()

plt.show()

In [ ]:
pd.set_option("display.max_rows", None)
print(amenity_combined["amenity"].value_counts())

In [ ]:
for col in amenity_combined.columns:
    print(col)

In [ ]:
amenity_combined.info()

In [ ]:
# checking types
pd.set_option("display.max_rows", None)
print(shops_combined["shop"].value_counts())  # Top shop types
print(leisure_combined["leisure"].value_counts())  # Top leisure types



In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
# Checking column names
print(list(shops_combined.columns))


In [ ]:
print(list(leisure_combined.columns))

## Osm data cleaning

#### Cleaning amenities data

In [ ]:
# Calculating the proportion of nonmissing values per column
non_missing_ratio = amenity_combined.notna().mean()

# Keeping only columns with at least 20% data
cols_keep = non_missing_ratio[non_missing_ratio >= 0.2].index
amenity_reduced = amenity_combined[cols_keep]
print(list(amenity_reduced.columns))

In [ ]:
amenity_cleaned = amenity_reduced.drop(columns=["backrest"]) # removing backrest aswell because that is not useful at all

In [ ]:
amenity_cleaned[["geometry", "amenity"]].isna().sum() # check for missing values.

In [ ]:
# amenities that are likely to influence rent prices
important_amenities = [
    # education
    "school", "university", "college", "kindergarten", "childcare", "training", "language_school",
    # health
    "pharmacy", "hospital", "clinic", "doctors", "dentist", 
    # admin
    "atm", "bank", "post_office", "courthouse", "townhall",
    # transport
    "parking", "parking_space", "bicycle_parking",
    "charging_station", "fuel", "taxi", "bus_station",
    # leisure
    "restaurant", "cafe", "bar", "pub", "fast_food", "cinema", "theatre",
    "arts_centre", "concert_hall", "nightclub",
    # public
    "community_centre", "social_facility", "library", "marketplace", "public_building",
    "coworking_space", "events_venue",
    # safety
    "police", "fire_station",
    # utilities
    "post_box", "parcel_locker", "recycling", "waste_disposal", "drinking_water"
]

In [ ]:
amenity_clean = amenity_cleaned[amenity_cleaned["amenity"].isin(important_amenities)].reset_index(drop=True)


In [ ]:
pd.set_option("display.max_rows", None)
print(amenity_clean["amenity"].value_counts())

In [ ]:
pd.reset_option("display.max_rows")

#### Cleaning shops data

In [ ]:
# Calculating the proportion of nonmissing values per column
non_missing_ratio_shops = shops_combined.notna().mean()

# Keeping only columns with at least 20% data
cols_keep_shops = non_missing_ratio_shops[non_missing_ratio_shops >= 0.2].index
shops_reduced = shops_combined[cols_keep_shops].copy()
print(list(shops_reduced.columns))

In [ ]:
shops_reduced[['geometry', 'addr:city', 'addr:housenumber', 'addr:postcode', 'addr:street', 'name', 'opening_hours', 'operator', 'phone', 'shop', 'website', 'wheelchair', 'addr:country', 'check_date', 'level']].isna().sum() # check for missing values.
# looking to see if these fields could be valuable

In [ ]:
shops_clean = shops_reduced.drop(columns=['addr:city', 'addr:housenumber', 'addr:postcode', 'addr:street', 'name', 'opening_hours', 'operator', 'phone', 'website', 'wheelchair', 'addr:country', 'check_date', 'level']) 
# removing other data. doesn't seem useful

In [ ]:
shops_clean

In [ ]:
weird_shops = ["yes", "no", "vacant"]  # basically NaN values
shops_clean = shops_clean[~shops_clean["shop"].isin(weird_shops)]

In [ ]:
shop_counts = shops_clean["shop"].value_counts()

# Keeping shop types that appear at least 3 times
shops_clean = shops_clean[shops_clean["shop"].isin(shop_counts[shop_counts >= 3].index)]

In [ ]:
pd.set_option("display.max_rows", None)
print(shops_clean["shop"].value_counts()) 

In [ ]:
pd.reset_option("display.max_rows")

#### Cleaning leisure data

In [ ]:
# Calculating the proportion of nonmissing values per column
non_missing_ratio_leisure = leisure_combined.notna().mean()

# Keeping only columns with at least 20% data
cols_keep_leisure = non_missing_ratio_leisure[non_missing_ratio_leisure >= 0.2].index
leisure_reduced = leisure_combined[cols_keep_leisure].copy()
print(list(leisure_reduced.columns))

In [ ]:
leisure_reduced[['geometry', 'access', 'leisure', 'sport', 'garden:type']].isna().sum() # check for missing values.
# looking to see if these fields could be valuable

In [ ]:
leisure_clean = leisure_reduced.drop(columns=[ 'access', 'sport', 'garden:type']) # removing other data. doesn't seem useful

In [ ]:
pd.set_option("display.max_rows", None)
print(leisure_clean["leisure"].value_counts()) 

In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
# this data requires no further cleaning 

## Osm data aggregation

#### Aggregating amenities data

In [ ]:
# making a dictionary for amenity groups
amenity_categories = {
    "education": ["school", "kindergarten", "childcare", "training", "language_school"],
    
    "higher_education": ["university", "college"],
    
    "healthcare": ["pharmacy", "clinic", "hospital", "doctors", "dentist"],
    
    "finance_admin": ["atm", "bank", "courthouse", "post_office", "townhall"],
    
    "transport": ["bus_station", "taxi", "fuel", "charging_station", "police", "fire_station", "bicycle_rental"],
    
    "parking": ["parking", "parking_space", "bicycle_parking"],
    
    "food_drink": ["restaurant", "cafe", "fast_food", "bar", "pub", "marketplace"],

    "community_social": ["community_centre", "social_facility", "library", "coworking_space","public_building", "events_venue", "arts_centre", "theatre", "concert_hall", "nightclub", "cinema"],
    
    "services_utilities": ["recycling", "waste_disposal", "drinking_water", "post_box", "parcel_locker"]}

In [ ]:
amenity_to_category = {
    amenity: category
    for category, amenities in amenity_categories.items()
    for amenity in amenities
}

In [ ]:
# Function to map an amenity to its group
amenity_clean["category"] = amenity_clean["amenity"].map(amenity_to_category)


In [ ]:
amenity_clean

In [ ]:
print(amenity_clean["category"].value_counts(dropna=False))

#### Aggregating shops data

In [ ]:
shop_categories = {
    "clothing": ["clothes", "shoes", "jewelry", "bag", "tailor", "second_hand"],
    "food_grocery": ["supermarket", "bakery", "butcher", "greengrocer", "convenience", "alcohol"],
    "health_beauty": ["hairdresser", "beauty", "optician", "medical_supply", "e-cigarette"],
    "home_furniture": ["furniture", "appliance", "doityourself", "hardware", "bed"],
    "electronics": ["electronics", "computer", "communication"],
    "sports_leisure": ["sports", "toys", "fishing"],
    "vehicles": ["car", "car_repair", "car_parts", "bicycle", "tyres"],
    "misc_services": ["kiosk", "locksmith", "dry_cleaning", "copyshop", "gift", "trade", "stationery", "craft", "baby_goods", "erotic", "general", "mall", "florist"],
    "other": ["books", "tobacco", "pet"]
}

In [ ]:
shop_to_category = {
    shop: category
    for category, shops in shop_categories.items()
    for shop in shops
}

In [ ]:
shops_clean["category"] = shops_clean["shop"].map(shop_to_category)

In [ ]:
print(shops_clean["category"].value_counts(dropna=False))


#### Aggregating leisure data

In [ ]:
leisure_categories = {
    "parks_green": ["garden", "park", "nature_reserve", "dog_park"],
    "sports_facilities": ["pitch", "sports_centre", "fitness_station", "fitness_centre", "track", "stadium", "sports_hall", "bowling_alley", "trampoline_park", "horse_riding", "ice_rink", "swimming_pool","disc_golf_course","swimming_area"],
    "play_recreation": ["playground", "indoor_play", "picnic_table", "bleachers", "outdoor_seating", "village_swing", "firepit", "sauna", "marina", "slipway"],
    "other": ["hackerspace", "tanning_salon"]
}

In [ ]:
leisure_to_category = {
    leisure: category
    for category, leisures in leisure_categories.items()
    for leisure in leisures
}

In [ ]:
leisure_clean["category"] = leisure_clean["leisure"].map(leisure_to_category)

In [ ]:
print(leisure_clean["category"].value_counts(dropna=False))

#### Saving the datasets

In [ ]:
amenity_clean.to_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/amenity_clean.gpkg", layer="amenity", driver="GPKG")
shops_clean.to_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/shop_clean.gpkg", layer="shop", driver="GPKG")
leisure_clean.to_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/leisure_clean.gpkg", layer="leisure", driver="GPKG")

# Analysis

In [ ]:
# import here so do not have to run the whole code again and results dont change since osm data might change in time

In [ ]:
gdf_amenity = gpd.read_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/amenity_clean.gpkg", layer="amenity")
gdf_shop = gpd.read_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/shop_clean.gpkg", layer="shop")
gdf_leisure = gpd.read_file("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_OSM/leisure_clean.gpkg", layer="leisure")

### Finding correlations between distance to services and price 

In [ ]:
# finding the closest node to each service and rental

In [ ]:
# nearest node to each rental
gdf_rent["nearest_node"] = ox.distance.nearest_nodes(G, gdf_rent.geometry.x, gdf_rent.geometry.y)

# nearest node to each service type
for gdf in [gdf_amenity, gdf_shop, gdf_leisure]:
    gdf["nearest_node"] = ox.distance.nearest_nodes(G, gdf.geometry.x, gdf.geometry.y)

In [ ]:
all_categories = (
    list(gdf_amenity["category"].unique()) +
    list(gdf_shop["category"].unique()) +
    list(gdf_leisure["category"].unique()))
print(all_categories)

In [ ]:
category_nodes = {}

# for filling in the dict
def add_category_nodes(gdf):
    for cat, group in gdf.groupby("category"):
        category_nodes[cat] = set(group["nearest_node"])

add_category_nodes(gdf_amenity)
add_category_nodes(gdf_shop)
add_category_nodes(gdf_leisure)

In [ ]:
category_distances = {}
for cat, nodeset in category_nodes.items():
    print(f"calculate network distances for category: {cat} ({len(nodeset)} sources)")
    category_distances[cat] = nx.multi_source_dijkstra_path_length(G, nodeset, weight='length')

In [ ]:
for cat, dist_dict in category_distances.items():
    gdf_rent[f"dist_{cat}"] = gdf_rent["nearest_node"].map(dist_dict)

In [ ]:
corr = gdf_rent.filter(like='dist_').assign(price=gdf_rent['price']).corr()
print(corr['price'].sort_values())

In [ ]:
# intresting correlations. rent prices near education services like schools seem to be lower and near higher education they seem to be higher.
# THIS IS WRONG BECAUSE RENTS ARE AUTOCORRELATED WITH EACHOTHER
# Toblers first law of geography : "everything is related to everything else, but near things are more related than distant things."

### Controlling for spacial autocorrelation

In [ ]:
# create spatial weights matrix. Meaning it will find 8 of the nearest rentals based on coordinates.
w = KNN.from_dataframe(gdf_rent, k=8) # assings weight 1 to the closest 8 and 0 to everyone else
w.transform = 'r'  # standardizes the weights. If is neighbour gets devided by "k". So 1/8 will be the weight of each neighbour

In [ ]:
# Calculate bivariate Moran's I for each distance variable
# what is  bivariate Moran's I
# a test that looks for spatial correlation between two variables while accounting for spatial autocorrelation
# IMPORTANT to find if higher/lower rents are nearer to some service categories
results = {}
for col in gdf_rent.filter(like='dist_').columns:
    moran_bv = Moran_BV(gdf_rent[col], gdf_rent['price'], w) # finding bivariate Moran's I between distances, rent and spacial weights
    results[col] = {
        'I': moran_bv.I,
        'p_value': moran_bv.p_sim,  # using permutation test to see if values are satistically signifficant
        'z_score': moran_bv.z_sim}

In [ ]:
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('I')
print(results_df)

In [ ]:
# significant relationships p < 0.05
# Still some relationships persist. Proximity to higher education and parks is associated with higher prices. Bigger distance to education and home furniture is associated with lower prices. 

In [ ]:
# spatial lag modeling to see if after controlling for spacial autocorrelation there still is a relationship.

In [ ]:
# selecting variables that showed a significant relationship
significant_vars = [
    'dist_parks_green',
    'dist_higher_education', 
    'dist_education',
    'dist_home_furniture']

In [ ]:
# preparing the data
# because ML_Lag wants the data in numpy arrays
X = gdf_rent[significant_vars].values # selects only distances that were significant in previous tests
y = gdf_rent['price'].values # gets prices

In [ ]:
# adding constant
X = np.column_stack([np.ones(len(X)), X]) # adding an intercept
var_names = ['constant'] + significant_vars # adding names to variables

In [ ]:
# spatial lag model to see if prices influence nearby prices
lag_model = ML_Lag(
    y, X, w=w,
    name_y='price',
    name_x=var_names,
    name_w='spatial_weights')

In [ ]:
print(lag_model.summary)

In [ ]:
# When controlling for spacial autocorrelation some of the signifficant results go away such as parks and home_furniture stores.
# Smaller distance to higher education still seems to increase prices and bigger distance form regular education seems to lower prices. Look at coefficient values.
# every 1000m form a higher education service your rent will decrease by 33€ 
# every 1000m you are from a regular education service your rent will be increase by 85€. 
# so it is good to be far from univercity buildings and close to regular schools.
# There also is a spacial correlation between prices themselves. If nearby apartments cost more, others will too.

In [ ]:
# These results seem werid. Especially the distance from regular schools increasing your price. It should be the other way.
# I will try to see if it can be controlled with city center being more expensive. Maybe most schools are far from the city center etc.

In [ ]:
city_center = Point(659247,6474311) # point on raekoja plats considered for city center.

In [ ]:
center_node = ox.distance.nearest_nodes(G, X=city_center.x, Y=city_center.y)
center_node

In [ ]:
dist_from_center = nx.single_source_dijkstra_path_length(G,center_node,weight='length')

In [ ]:
gdf_rent["dist_center"] = gdf_rent["nearest_node"].map(dist_from_center)

In [ ]:
X_plus_center = gdf_rent[['dist_parks_green', 'dist_higher_education', 
                           'dist_education', 'dist_home_furniture', 
                           'dist_center']].values
X_plus_center = np.column_stack([np.ones(len(X_plus_center)), X_plus_center])


In [ ]:
lag_with_center = ML_Lag(
    y, X_plus_center, w=w,
    name_y='price',
    name_x=['constant', 'dist_parks_green', 'dist_higher_education', 
            'dist_education', 'dist_home_furniture', 'dist_center'],
    name_w='spatial_weights') # fiting a spacial lag model

In [ ]:
print("MODEL WITH CITY CENTER DISTANCE")
print(lag_with_center.summary)

In [ ]:
# The regular education weirdness still persists. For this there might be multiple explanations. Rentals being older or in worse condition near regular education or people with kids don't rent they buy etc.

### Walkability analysis

In [ ]:
dist_cols = [col for col in gdf_rent.columns if col.startswith("dist_")]

In [ ]:
gdf_rent[dist_cols] = gdf_rent[dist_cols].clip(upper=2000) # capping distances to 2km

In [ ]:
tau = 500 # decay constant

In [ ]:
# dictionary of weights for daily walkability
weights = {
    # important
    "food_grocery": 0.12,
    "food_drink": 0.10,
    "healthcare": 0.10,
    "education": 0.08,
    "higher_education": 0.06,
    "transport": 0.10,
    "parks_green": 0.08,
    "play_recreation": 0.05,
    # maybe
    "sports_facilities": 0.05,
    "sports_leisure": 0.04,
    "community_social": 0.04,
    "finance_admin": 0.03,
    "services_utilities": 0.03,
    "misc_services": 0.02,
    "health_beauty": 0.02,
    # once in a while
    "home_furniture": 0.01,
    "clothing": 0.01,
    "vehicles": 0.01,
    "electronics": 0.01,
    "parking": 0.01,
    "other": 0.01
}

In [ ]:
for cat in weights.keys():
    dist_col = f"dist_{cat}"
    if dist_col in gdf_rent.columns:
        gdf_rent[f"score_{cat}"] = np.exp(-gdf_rent[dist_col] / tau)

In [ ]:
# each score multiplied by its weight, then sum
weighted_scores = []
for cat, w in weights.items():
    col = f"score_{cat}"
    if col in gdf_rent.columns:
        weighted_scores.append(gdf_rent[col] * w)

gdf_rent["walkability_score_weighted"] = np.sum(weighted_scores, axis=0)

In [ ]:
gdf_rent["walkability_index"] = 100 * (
    (gdf_rent["walkability_score_weighted"] - gdf_rent["walkability_score_weighted"].min()) /
    (gdf_rent["walkability_score_weighted"].max() - gdf_rent["walkability_score_weighted"].min())
)

In [ ]:
sns.histplot(gdf_rent["walkability_index"], kde=True)

In [ ]:
gdf_rent.plot(column="walkability_index", cmap="viridis", legend=True)

### Amenity diversity score

In [ ]:
#gdf_rent.columns

In [ ]:
radius = 800 # radius of search for services

In [ ]:
# counting how many categories of service are within 800 m along the network
category_cols = [col for col in gdf_rent.columns if col.startswith("dist_")]

In [ ]:
# boolean
within_radius = gdf_rent[category_cols] <= radius

In [ ]:
# counting number of categories within radius
gdf_rent["amenity_diversity_net"] = within_radius.sum(axis=1)

In [ ]:
# normalization
gdf_rent["amenity_diversity_net_norm"] = gdf_rent["amenity_diversity_net"] / 21

In [ ]:
sns.scatterplot(data=gdf_rent, x="amenity_diversity_net_norm", y="price")
plt.xlabel("Amenity diversity")
plt.ylabel("price")
plt.show()

### Plots for data visualization

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(12, 10))
gdf_rent.plot(
    column="walkability_index",
    cmap="OrRd",
    legend=True,
    markersize=20,
    alpha=0.8,
    ax=ax
)
#gdf_place_web.boundary.plot(ax=ax, edgecolor="black", linewidth=2)


ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs=gdf_rent.crs)

ax.set_title("Walkability Index", fontsize=16)
ax.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
gdf_rent.plot(
    column="amenity_diversity_net",
    cmap="OrRd",
    legend=True,
    markersize=20,
    alpha=0.8,
    ax=ax
)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.VoyagerNoLabels, crs=gdf_rent.crs)

ax.set_title("Servicecategories within 800 m", fontsize=16)
ax.axis('off')
plt.show()


### Saving the engineered features

In [ ]:
gdf_rent.to_file("gdf_rent_engineered.gpkg", layer="gdf_rent_engineered", driver="GPKG")

In [ ]:
#gdf_rent_features = gpd.read_file("gdf_rent_engineered.gpkg", layer="gdf_rent_engineered")

In [ ]:
#gdf_rent_features.head()

# Random forest model building

## RF simple without any engineered features

### Reload the data without any engineered features

In [ ]:
df = pd.read_csv("C:/Users/Dan/Documents/GitHub/KV-scraper-SAT/Cleaned_csvs/listings_cleaned_tartu.csv", sep=";") 
gdf_rent = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
gdf_rent = gdf_rent.to_crs(epsg=3301)

In [ ]:
print(gdf_rent.columns.tolist())

### Making data ready

In [ ]:
gdf_rent['condition'].unique()

In [ ]:
condition_mapping = {
    'Uus': 5,                        
    'San. remont tehtud': 4,         
    'Renoveeritud': 4,               
    'Valmis': 3.5,                   
    'Heas korras': 3,                
    'Keskmine': 2,                          
    'Vajab san. remonti': 1,
    'Vajab renoveerimist': 0.5,}

In [ ]:
# map condition scores to data
gdf_rent['condition_score'] = gdf_rent['condition'].map(condition_mapping)

In [ ]:
print(gdf_rent.isnull().sum()[gdf_rent.isnull().sum() > 0])

In [ ]:
# fill missing values if any with median
gdf_rent['condition_score'] = gdf_rent['condition_score'].fillna(gdf_rent['condition_score'].median())

In [ ]:
print(gdf_rent.isnull().sum()[gdf_rent.isnull().sum() > 0])

In [ ]:
# removing non-feature columns
exclude_cols = ['geometry', 'latitude', 'longitude', 'price', 'url', 
                'id', 'date_scraped', 'condition']

In [ ]:
# getting all columns that will be used
feature_cols = [col for col in gdf_rent.columns 
                if col not in exclude_cols 
                and gdf_rent[col].dtype in ['int64', 'float64']]

### Train test split

In [ ]:
X = gdf_rent[feature_cols].copy()
y = gdf_rent['price'].copy()

In [ ]:
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
# fill missing values if any with median
if X.isnull().any().any():
    X = X.fillna(X.median())

In [ ]:
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

### Model training 

In [ ]:
#Random Forest model
rf_model = RandomForestRegressor(n_estimators=1000,max_depth=20,min_samples_split=2,random_state=42,n_jobs=-1)

In [ ]:
rf_model.fit(X_train, y_train)


### Crossvalidation

In [ ]:
cv_scores = cross_val_score(rf_model, X_train, y_train,cv=10,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
# converting negative MSE to RMSE
cv_rmse = np.sqrt(-cv_scores)

In [ ]:
for i, rmse in enumerate(cv_rmse):
    print(f"  Fold {i+1}: {rmse:.2f} EUR")

In [ ]:
print(f"\nAverage CV RMSE: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f} EUR")

### Test set test

In [ ]:
# make predictions
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

# calculating metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# calculating metrics 
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train")
print(f"  RMSE: {train_rmse:.2f} EUR")
print(f"  MAE:  {train_mae:.2f} EUR")
print(f"  R2:   {train_r2:.4f}")

print("Test")
print(f"  RMSE: {test_rmse:.2f} EUR")
print(f"  MAE:  {test_mae:.2f} EUR")
print(f"  R2:   {test_r2:.4f}")

In [ ]:
# As expected the results are bad.

## RF with engineered features

### Data loading

In [ ]:
gdf_rent_features = gpd.read_file("gdf_rent_engineered.gpkg", layer="gdf_rent_engineered")

In [ ]:
print(gdf_rent_features.columns.tolist())

### Making data ready

In [ ]:
condition_mapping = {
    'Uus': 5,                        
    'San. remont tehtud': 4,         
    'Renoveeritud': 4,               
    'Valmis': 3.5,                   
    'Heas korras': 3,                
    'Keskmine': 2,                          
    'Vajab san. remonti': 1,
    'Vajab renoveerimist': 0.5,}

In [ ]:
# map condition scores to data
gdf_rent_features['condition_score'] = gdf_rent_features['condition'].map(condition_mapping)

In [ ]:
# fill missing values if any with median
gdf_rent_features['condition_score'] = gdf_rent_features['condition_score'].fillna(gdf_rent_features['condition_score'].median())

In [ ]:
# # removing non-feature columns
exclude_cols = [
    'id', 'url', 'latitude', 'longitude', 'geometry', 'price', 
    'condition', 'nearest_node']

In [ ]:
score_cols = [c for c in gdf_rent_features.columns if c.startswith("score_")] # removing all columns with score_ in name because they are similar to distance
exclude_cols.extend(score_cols)

In [ ]:
# getting all columns that will be used
feature_cols = [col for col in gdf_rent_features.columns 
                if col not in exclude_cols 
                and gdf_rent_features[col].dtype in ['int64', 'float64', 'Int64', 'Float64']]

### Train test split

In [ ]:
X = gdf_rent_features[feature_cols].copy()
y = gdf_rent_features['price'].copy()

In [ ]:
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
if X.isnull().any().any():
    X = X.fillna(X.median())

In [ ]:
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
print(X.columns.tolist())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=200,max_depth=15,min_samples_split=2,random_state=42,n_jobs=-1)

In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
cv_scores = cross_val_score(rf_model, X_train, y_train,cv=10,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
cv_rmse = np.sqrt(-cv_scores)
print(f"CV RMSE: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f} EUR\n")

In [ ]:
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"\nTrain: RMSE={train_rmse:.2f} EUR | R²={train_r2:.4f}")
print(f"Test:  RMSE={test_rmse:.2f} EUR | R²={test_r2:.4f} | MAE={test_mae:.2f} EUR")

In [ ]:
# predicts still extremely bad.

In [ ]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)
print(feature_importance.head(20).to_string(index=False))

In [76]:
condition_mapping = {
    'Uus': 5,
    'San. remont tehtud': 4,
    'Renoveeritud': 4,
    'Valmis': 3.5,
    'Heas korras': 3,
    'Keskmine': 2,
    'Vajab san. remonti': 1,
    'Vajab renoveerimist': 0.5,}

In [77]:
gdf['condition_score'] = (gdf['condition'].map(condition_mapping))

In [78]:
gdf['condition_score'] = gdf['condition_score'].fillna(gdf['condition_score'].median())

In [79]:
exclude_cols = [
    "id", "url", "condition", "geometry", 'latitude', 'longitude',
    "price", "nearest_node"] # exluding 

In [80]:
dist_cols = [c for c in gdf.columns if c.startswith("dist_") and c != "dist_center"]  # keep dist_center for kriging
score_cols = [c for c in gdf.columns if c.startswith("score_")] # excluding because krigging will evaluate these

In [81]:
exclude_cols.extend(dist_cols)
exclude_cols.extend(score_cols)

In [82]:
feature_cols = [
    c for c in gdf.columns
    if c not in exclude_cols
    and gdf[c].dtype in ["float64", "int64", "Int64", "Float64"]]

In [83]:
gdf['area_condition'] = gdf['area_sqm'] * gdf['condition_score']
gdf['rooms_floor']    = gdf['rooms'] * gdf['floor']

feature_cols += ['area_condition', 'rooms_floor']

In [84]:
X = gdf[feature_cols].copy()
y = gdf["price"].copy()
X = X.fillna(X.median())

In [85]:
print(X.columns.tolist())

['area_sqm', 'floor', 'total_floors', 'rooms', 'dist_center', 'walkability_score_weighted', 'walkability_index', 'amenity_diversity_net', 'amenity_diversity_net_norm', 'condition_score', 'area_condition', 'rooms_floor']
